*written by: Julia Knoblauch, Jana Vihs, Annika Boer, Kai Ingo Schewina*

# Explainable Recommendation Systems for Energy-Efficient Smart Home

requirements.txt

# To do:
- Annika: xgboost, lime liste von outputs
- Jana: GAMMLI - output, recommendation after weather & xai
- Julia: activity eval fertig machen, usage eval sobald hourly, shap liste outputs
- Kai: usage hourly, weather dass pipeline runnt

Weiteres:
- EDA: auf germain paper anpassen und für weather updaten
- evaluierung runnen für (alle) households
- explanability agent erstellen: activity_agent & usage agent zusammenführen für recommendation
- hyperparameter tuning: über performance evaluation agent
- (other datasets)

Sammlung Hinweise:
- warning bei usage/load wenn man evaluierung runnt zu format --> beheben?
(warning bei auc/load: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().)
- xgb: use_label_encoder=False eingestellt damit warning weg geht: problem?
- explainability evaluation atm only test first 3 rows (set to len(X_test)) for final eval

## Abstract
should actually be before TOC and not numbered

## Table of contents

1. Introduction:
    - Problem setting: ?
    - Current Recommender System --> agents + Bild (Jana)

2. Literature Review (Kai) --> Tabelle
    - Explainable AI (XAI): review zusammenfassen und aktualisieren
    - Recommender Systems in SH: Fokus
    - Explainability in RS: Fokus
3. Methodology (i.e., the ML techniques that you apply)
    - Recommender System??
    - Algorithms XAI
        - Taxonomy (Bild) (Annika)
        - Model-intrinsic (Annika)
        - Model-agnostic (Machine Learning Interpretability: A Survey on Methods and Metrics)
            - LIME (Annika)
            - SHAP (Julia)
            - other approaches?
        - vorgestellte Tabelle aus Präsi formaler --> Vergleich
    - Algorithms prediction (Jana, Formel für finales Modell, mit paper)
        xgboost, adaboost, knn, logit, rf, glm?

4. Experimental Design (including EDA, data organization, performance measures, etc.)
    - data organization (+ EDA von denen auf Wetterdaten + NAs Wetterdaten)
        - agents system (Bild) (Jana)
        - restriction to 2 households
        - with & without weather data (EDA, Struktur, Herkunft) (Kai)
    - Evaluation
        - Performance (Kai?)
        - Explainability (feature importance) (Julia, Annika)
5. Results
    - Performance (Kai?)
    - Explainability (Julia, Annika)
    - Decision for model & XAI model
    - Final result of RS: Recommendation + Explanation (Jana)
6. Discussion
    - ...
    - Contributions
    - Limitations
    - Implications
    - Recommendations
    - Future Research: Neural Networks

e.g. include:
    - no user
    - runtime problems
        - api weather data 
        - 4 models, 20 households
        - Beispiel, wie lange es dauern würde, eine recommendation zu laden (aber nur 1x am Tag)

7. Conclusion
References

## 1. Introduction

## 2. Literature Review
https://christophm.github.io/interpretable-ml-book/

### Explainable AI (XAI): review zusammenfassen und aktualisieren
### Recommender Systems in SH: Fokus
#### Explainability in RS: Fokus

## 3. Methodology

### 3.1 Recommender System

### 3.2 XAI Alogorithms
### Taxonomy?
### Model intrinsic
Logit as benchmark?, GAMMLI, Random Forest
### Model Agnostic Approaches

### LIME
To do:
Is mode= 'regression' correct?

xgboost: resources to implement
https://github.com/marcotcr/lime/issues/334
https://github.com/adriamoya/xgboost_default_companies/blob/master/main.py

### SHAP
Aktueller Stand:
logit: kernel (linearExplainer möglich aber dann anderes array als output für shapley values)
ada: kernel (eigene implementierung in treee shap möglich aber internetversion hat bei mir auf die schnelle nicht funktioniert)
knn: kernel, mask doesnt work
rf: tree without predict proba
xgb: tree

Möglichkeit:
X_test ganz berechnen: aber glaub gar nicht so viel schneller und
Zeit wäre verzerrt

Aktuell implementiert für KernelExplainer: vorher kmeans weil viel schneller
aber keine info zu tradeoff und keine wirklichen recommendations wie k= gesetzt werden sollte


### Other approaches to research:
https://www.linkedin.com/posts/hima-lakkaraju-b130217_himachilpptx-activity-6814735906092703744-fOYe/
https://interpretable-ml-class.github.io/
https://github.com/anguyen8/XAI-papers
https://towardsdatascience.com/the-how-of-explainable-ai-explainable-modelling-55c8c43d7bed
https://github.com/SeldonIO/alibi
https://github.com/pbiecek/xai_resources
https://github.com/interpretml/interpret

**MUSE**
chrome-extension://efaidnbmnnnibpcajpcglclefindmkaj/viewer.html?pdfurl=https%3A%2F%2Fcs.stanford.edu%2Fpeople%2Fjure%2Fpubs%2Fexplanations-aies19.pdf&clen=1929781&chunk=true

**BETA**
https://arxiv.org/pdf/1707.01154.pdf

**Anchors**
https://homes.cs.washington.edu/~marcotcr/aaai18.pdf
https://github.com/marcotcr/anchor

### 3.3 Prediction Algorithms
**XGBoost**

**AdaBoost**

**K-Nearest-Neighbour**

**Logistic Regression**

**Random Forest**

### XGBoost

Achtung: depreciation warning for XGB
--> label encoder = False eingestellt
--> noch mehr verändern?

## 6. Discussion and Limitations

## 7. Conclusion

## References